In [ ]:
!pip install -q ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.4 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -q ultralytics pandas opencv-python pillow


In [ ]:
CLASS_MAP = {
    "bolt": 0,
    "locatingpin": 1,
    "nut": 2,
    "washer": 3
}




In [ ]:
DATASET_PATH = "/content/drive/MyDrive/SolidWorksAI"


In [ ]:
import os
os.listdir(DATASET_PATH)


['sample_submission.csv',
 'train_bboxes.csv',
 'train_labels.csv',
 'test',
 'train']

In [ ]:
import os

BASE_YOLO = "/content/datasets"

os.makedirs(f"{BASE_YOLO}/images/train", exist_ok=True)
os.makedirs(f"{BASE_YOLO}/images/val", exist_ok=True)
os.makedirs(f"{BASE_YOLO}/labels/train", exist_ok=True)
os.makedirs(f"{BASE_YOLO}/labels/val", exist_ok=True)


In [ ]:
import random, shutil

train_img_path = f"{DATASET_PATH}/train"
images = os.listdir(train_img_path)
random.shuffle(images)

split = int(0.8 * len(images))
train_imgs = images[:split]
val_imgs = images[split:]

for img in train_imgs:
    shutil.copy(f"{train_img_path}/{img}", f"{BASE_YOLO}/images/train")

for img in val_imgs:
    shutil.copy(f"{train_img_path}/{img}", f"{BASE_YOLO}/images/val")


In [ ]:
import pandas as pd
from PIL import Image

df = pd.read_csv(f"{DATASET_PATH}/train_bboxes.csv")

def yolo_bbox(row, w, h):
    xc = (row.x_min + row.x_max) / 2 / w
    yc = (row.y_min + row.y_max) / 2 / h
    bw = (row.x_max - row.x_min) / w
    bh = (row.y_max - row.y_min) / h
    return xc, yc, bw, bh

for img_name in df.image_name.unique():
    img_path = f"{train_img_path}/{img_name}"
    img = Image.open(img_path)
    w, h = img.size

    rows = df[df.image_name == img_name]

    label_dir = "train" if img_name in train_imgs else "val"
    label_file = f"{BASE_YOLO}/labels/{label_dir}/{img_name.replace('.png','.txt')}"

    with open(label_file, "w") as f:
        for _, r in rows.iterrows():
            cls = CLASS_MAP[r["class"]]
            bbox = yolo_bbox(r, w, h)
            f.write(f"{cls} {' '.join(map(str, bbox))}\n")


In [ ]:
%%writefile solidworks.yaml
path: /content/datasets
train: images/train
val: images/val

names:
  0: bolt
  1: locatingpin
  2: nut
  3: washer


Writing solidworks.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

model.train(
    data="solidworks.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device=0
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=solidworks.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7de37278f020>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")

results = model(f"{DATASET_PATH}/test", conf=0.25)



WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2000 /content/drive/MyDrive/SolidWorksAI/test/0040313aa7c7478c8ca264bf573c53fe.png: 640x640 1 bolt, 3 locatingpins, 16.4ms
image 2/2000 /content/drive/MyDrive/SolidWorksAI/test/0080e45f4375443b96ee81dc04075117.png: 640x640 2 bolts, 1 washer, 16.3ms
image 3/2000 /content/drive/MyDrive/SolidWorksAI/test/0086da1a1914469caa4d042f8e4e94ef.png: 640x640 1 washer, 16.3ms
image 4/2000 /content/drive/MyDrive/SolidWorksAI/test/009ccd5b64a043e0afbae0f840f7cf

In [ ]:
from collections import Counter

rows = []

for r in results:
    counts = Counter(r.boxes.cls.cpu().numpy().astype(int))
    rows.append({
        "image_name": r.path.split("/")[-1],
        "bolt": counts.get(0, 0),
        "locatingpin": counts.get(1, 0),
        "nut": counts.get(2, 0),
        "washer": counts.get(3, 0)
    })

submission = pd.DataFrame(rows).sort_values("image_name")


In [ ]:
OUTPUT_PATH = "/content/drive/MyDrive/solidworks_submission.csv"
submission.to_csv(OUTPUT_PATH, index=False)

print("✅ Submission saved to:", OUTPUT_PATH)


✅ Submission saved to: /content/drive/MyDrive/solidworks_submission.csv
